In [1]:
!ollama pull mistral


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         
pulling 491dfa501e59... 100% ▕████████████████▏  801 B                         
pulling ed11eda7790d... 100% ▕████████████████▏   30 B                         
pulling 42347cd80dc8... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [2]:
!ollama list


NAME                       ID              SIZE      MODIFIED      
mistral:latest             f974a74358d6    4.1 GB    4 seconds ago    
jayati:latest              01eddb86861b    2.0 GB    2 weeks ago      
codegemma:latest           0c96700aaada    5.0 GB    2 weeks ago      
starcoder2:15b             21ae152d49e0    9.1 GB    2 weeks ago      
tinyllama:latest           2644915ede35    637 MB    2 weeks ago      
doctor:latest              e54af4a2c294    2.0 GB    2 weeks ago      
oldlady:latest             17c065d17fd9    2.0 GB    2 weeks ago      
qwen2.5:latest             845dbda0ea48    4.7 GB    3 weeks ago      
nomic-embed-text:latest    0a109f422b47    274 MB    6 weeks ago      
llama3.2:latest            a80c4f17acd5    2.0 GB    6 weeks ago      


In [3]:
!ollama show mistral


  Model
    architecture        llama    
    parameters          7.2B     
    context length      32768    
    embedding length    4096     
    quantization        Q4_0     

  Parameters
    stop    "[INST]"     
    stop    "[/INST]"    

  License
    Apache License               
    Version 2.0, January 2004    



In [4]:
pip install yfinance


Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install yfinance


In [6]:
import yfinance as yf
import pandas as pd
import numpy as np

from llama_index.core.readers import SimpleDirectoryReader
from llama_index.readers.file import PagedCSVReader
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

import chromadb
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore


In [7]:
!pip install llama-index pandas numpy yfinance chromadb


In [32]:
import yfinance as yf

def get_financial_ratios(ticker):
    try:
        stock = yf.Ticker(ticker)

        # Fetch financial statements
        financials = stock.financials
        balance_sheet = stock.balance_sheet
        cashflow = stock.cashflow

        # Debugging: Print available keys
        print("Available financials keys:", financials.index)
        print("Available balance sheet keys:", balance_sheet.index)
        print("Available cash flow keys:", cashflow.index)

        # Function to safely retrieve values
        def get_value(df, key):
            if key in df.index:
                return df.loc[key].iloc[0]  # Use .iloc[0] to get the latest value
            return None

        # Extract required values safely
        revenue = get_value(financials, "Total Revenue")
        net_income = get_value(financials, "Net Income")
        total_assets = get_value(balance_sheet, "Total Assets")
        total_liabilities = get_value(balance_sheet, "Total Liabilities Net Minority Interest")
        stockholders_equity = get_value(balance_sheet, "Stockholders Equity")  # ✅ FIXED KEY
        invested_capital = get_value(balance_sheet, "Invested Capital")  # ✅ FIXED KEY
        ebit = get_value(financials, "EBIT")
        operating_cashflow = get_value(cashflow, "Operating Cash Flow")  # ✅ FIXED KEY
        total_debt = get_value(balance_sheet, "Total Debt")  # ✅ FIXED KEY
        current_assets = get_value(balance_sheet, "Current Assets")
        current_liabilities = get_value(balance_sheet, "Current Liabilities")

        # Compute new ratios with fallbacks
        ratios = {
            "ROIC (%)": (net_income / invested_capital) * 100 if net_income and invested_capital else "Data Unavailable",
            "ROA (%)": (net_income / total_assets) * 100 if net_income and total_assets else "Data Unavailable",
            "Debt-to-Equity": total_liabilities / stockholders_equity if total_liabilities and stockholders_equity else "Data Unavailable",
            "Current Ratio": current_assets / current_liabilities if current_assets and current_liabilities else "Data Unavailable",
            "EBIT Margin (%)": (ebit / revenue) * 100 if ebit and revenue else "Data Unavailable",
            "Operating Cash Flow to Debt": operating_cashflow / total_debt if operating_cashflow and total_debt else "Data Unavailable"
        }

        return ratios

    except Exception as e:
        return {"Error": str(e)}

# Example usage
ticker = input("Enter the stock ticker (e.g., AAPL, TSLA): ")
print("\nFetching financial data...\n")
ratios = get_financial_ratios(ticker)
print("\nFinancial Ratios:")
for key, value in ratios.items():
    print(f"{key}: {value}")


Enter the stock ticker (e.g., AAPL, TSLA):  AAPL



Fetching financial data...

Available financials keys: Index(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
       'EBIT', 'Net Interest Income', 'Interest Expense', 'Interest Income',
       'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Total Operating Income As Reported',
       'Diluted Average Shares', 'Basic Average Shares', 'Diluted EPS',
       'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Net Income',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Income Expense', 'Other Non Operating Income Expenses',
       'Net Non Operating Interest Income Expense',
       'Interest Expense Non Operating',

In [33]:
def analyze_ratios_with_llm(ratios):
    prompt = f"""
    Given the following financial ratios, analyze the company's financial health and provide an investment recommendation:
    
    - ROIC: {ratios.get("ROIC (%)", "N/A")}%
    - ROA: {ratios.get("ROA (%)", "N/A")}%
    - Debt-to-Equity: {ratios.get("Debt-to-Equity", "N/A")}
    - Current Ratio: {ratios.get("Current Ratio", "N/A")}
    - EBIT Margin: {ratios.get("EBIT Margin (%)", "N/A")}%
    - Operating Cash Flow to Debt: {ratios.get("Operating Cash Flow to Debt", "N/A")}

    Based on these metrics, should an investor consider buying, holding, or selling this stock? Provide a short justification.
    """

    # Configure the LLM
    llm = Ollama(model="mistral")

    try:
        response = llm.complete(prompt)
        return response
    except Exception as e:
        return f"Error generating analysis: {str(e)}"


In [34]:
def main():
    ticker = input("Enter the stock ticker (e.g., AAPL, TSLA): ").strip().upper()
    
    print("\nFetching financial data...")
    ratios = get_financial_ratios(ticker)
    
    if "Error" in ratios:
        print("Error fetching data:", ratios["Error"])
        return

    print("\nFinancial Ratios:")
    for key, value in ratios.items():
        print(f"{key}: {value}")

    print("\nAnalyzing investment potential using LLM...")
    recommendation = analyze_ratios_with_llm(ratios)

    print("\nInvestment Recommendation:")
    print(recommendation)


In [37]:
if __name__ == "__main__":
    main()


Enter the stock ticker (e.g., AAPL, TSLA):  AAPL



Fetching financial data...
Available financials keys: Index(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
       'EBIT', 'Net Interest Income', 'Interest Expense', 'Interest Income',
       'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Total Operating Income As Reported',
       'Diluted Average Shares', 'Basic Average Shares', 'Diluted EPS',
       'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Net Income',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Income Expense', 'Other Non Operating Income Expenses',
       'Net Non Operating Interest Income Expense',
       'Interest Expense Non Operating', 